In [ ]:
import PyPDF2
import pandas as pd
import re
import matplotlib.pyplot as plt


In [ ]:
# Read the PDF file
pdf_path = 'Path/to/pdf/file.pdf'
pdf_reader = PyPDF2.PdfReader(pdf_path)
print(pdf_reader)

## Parse PDF Content and Create Dataframe 

In [ ]:
# Helper function to determine if a string represents a date
def is_date(string):
    try:
        # Convert the first 5 characters to a date format
        pd.to_datetime(string[:5], format='%m/%d')
        return True
    except ValueError:
        return False

# Helper function to check if a string is a valid amount
def is_amount(string):
    try:
        #convert the string to a float after removing common non-numeric characters
        float(string.replace(',', '').replace('$', '').strip())
        return True
    except ValueError:
        return False

# Function to clean the line by removing unnecessary parts based on section headers
def clean_line(line):
    for header in section_headers:
        if header in line:
            # Split the line at the header and only keep the part before the header
            line = line.split(header)[0].strip()
    return line

# State abbreviations with asterisks to identify potential amounts
state_abbreviations = [
    "*AL", "*AK", "*AZ", "*AR", "*CA", "*CO", "*CT", "*DE", "*FL", "*GA", "*HI", "*ID", "*IL", "*IN", "*IA", "*KS", "*KY", "*LA",
    "*ME", "*MD", "*MA", "*MI", "*MN", "*MS", "*MO", "*MT", "*NE", "*NV", "*NH", "*NJ", "*NM", "*NY", "*NC", "*ND", "*OH", "*OK",
    "*OR", "*PA", "*RI", "*SC", "*SD", "*TN", "*TX", "*UT", "*VT", "*VA", "*WA", "*WV", "*WI", "*WY", "*DC", "*AS", "*GU", "*MP",
    "*PR", "*VI", "*UM"
]

# Section headers to identify and split lines appropriately
section_headers = [
    "Other Withdrawals", "Service Charges", "Electronic Payments (continued)",
    "Subtotal:", "Other Credits", "begin", "POSTING DATE DESCRIPTION AMOUNT",
    "STATEMENT OFACCOUNT"
]

# Initialize lists for storing data
dates = []
merchants = []
categories = []
accounts = []
original_statements = []
notes = []
amounts = []

# Extract text from each page and remove footers (in this case, the footer starts with the bank phone number)
pdf_text = ""
for page in pdf_reader.pages:
    page_text = page.extract_text()
    # Remove footer text using regex
    page_text = re.sub(r'Call\s*1-800-937-2000.*', '', page_text)
    pdf_text += page_text

# Split text into lines
lines = pdf_text.split('\n')
print("Total Lines Extracted:", len(lines))

# First pass to clean and split lines
cleaned_lines = []
for line in lines:
    # Split line if it contains any section header
    if any(header in line for header in section_headers):
        for header in section_headers:
            if header in line:
                parts = line.split(header)
                cleaned_lines.append(parts[0].strip())
                if len(parts) > 1:
                    cleaned_lines.append(header + parts[1].strip())
    else:
        cleaned_lines.append(line)

# Second pass to process the cleaned lines
current_transaction_lines = []
current_date = None
for i, line in enumerate(cleaned_lines):
    if "DAILY BALANCE SUMMARY" in line:
        break  # Stop parsing when the DAILY BALANCE SUMMARY section is reached

    # Check if the line starts with a date
    if is_date(line[:5]):
        if current_transaction_lines:
            # Process the previous transaction
            transaction = " ".join(current_transaction_lines)
            last_line = clean_line(current_transaction_lines[-1].strip())

            # Find amount after state abbreviations
            amount = None
            for state in state_abbreviations:
                if state in last_line:
                    parts = last_line.split(state)
                    if len(parts) > 1:
                        potential_amount = parts[1].strip().split()[0]
                        if is_amount(potential_amount):
                            amount = potential_amount
                            break

            if not amount and last_line.split():
                # Try to find amount in the last part of the last line
                potential_amount = last_line.split()[-1]
                if is_amount(potential_amount):
                    amount = potential_amount

            if not amount:
                # Check if the amount is on a separate line after the state line
                for j in range(1, 4):  # Look ahead up to 3 lines
                    if i + j < len(cleaned_lines):
                        potential_split = cleaned_lines[i + j].strip().split()
                        if potential_split:
                            potential_amount = potential_split[-1]
                            if is_amount(potential_amount):
                                amount = potential_amount
                                break

            if amount:
                amounts.append(amount)
            else:
                amounts.append('*')  # Flag missing amount with an asterisk

            original_statements.append(transaction)
            merchants.append(transaction)
            categories.append("Category Placeholder")
            accounts.append("Account Placeholder")
            notes.append("")
            if not amount:
                dates.append('1234')  # Add placeholder for missing date
            else:
                dates.append(current_date)

            current_transaction_lines = []

        # Start a new transaction
        current_date = line[:5]
        current_transaction_lines.append(line)
    else:
        if "Subtotal:" in line:
            # Split the line at 'Subtotal:' and handle separately
            split_line = line.split("Subtotal:")
            current_transaction_lines.append(split_line[0].strip())
            if len(split_line) > 1:
                cleaned_lines[i] = "Subtotal:" + split_line[1].strip()
            else:
                cleaned_lines[i] = "Subtotal:"
            continue

        for header in section_headers:
            if header in line:
                # Split the line at the section header
                split_line = line.split(header)
                current_transaction_lines.append(split_line[0].strip())
                cleaned_lines[i] = header + split_line[1]  # Modify the current line to only contain the header and the rest
                break
        else:
            current_transaction_lines.append(line)

# Process the last transaction if any
if current_transaction_lines:
    transaction = " ".join(current_transaction_lines)
    last_line = clean_line(current_transaction_lines[-1].strip())

    # Find amount after state abbreviations
    amount = None
    for state in state_abbreviations:
        if state in last_line:
            parts = last_line.split(state)
            if len(parts) > 1:
                potential_amount = parts[1].strip().split()[0]
                if is_amount(potential_amount):
                    amount = potential_amount
                    break

    if not amount and last_line.split():
        # Try to find amount in the last part of the last line
        potential_amount = last_line.split()[-1]
        if is_amount(potential_amount):
            amount = potential_amount

    if not amount:
        # Check if the amount is on a separate line after the state line
        for j in range(1, 4):  # Look ahead up to 3 lines
            if i + j < len(cleaned_lines):
                potential_split = cleaned_lines[i + j].strip().split()
                if potential_split:
                    potential_amount = potential_split[-1]
                    if is_amount(potential_amount):
                        amount = potential_amount
                        break

    if amount:
        amounts.append(amount)
    elif amount is None:
        amounts.append('*')  # Flag missing amount with an asterisk
        dates.append('1234')  # Add placeholder for missing date
    else:
        dates.append(current_date)

    original_statements.append(transaction)
    merchants.append(transaction)
    categories.append("Category Placeholder")
    accounts.append("Account Placeholder")
    notes.append("")

# Ensure all lists are the same length by adding placeholders where necessary
max_length = max(len(dates), len(merchants), len(categories), len(accounts), len(original_statements), len(notes), len(amounts))

while len(dates) < max_length:
    dates.append('1234')
while len(merchants) < max_length:
    merchants.append('Placeholder')
while len(categories) < max_length:
    categories.append("Category Placeholder")
while len(accounts) < max_length:
    accounts.append("Account Placeholder")
while len(original_statements) < max_length:
    original_statements.append('Placeholder')
while len(notes) < max_length:
    notes.append('')
while len(amounts) < max_length:
    amounts.append('*')

# Print lengths of all lists for verification
print(len(dates), len(merchants), len(categories), len(accounts), len(original_statements), len(notes), len(amounts))


In [ ]:

# Create DataFrame
data = {
    'Date': dates,
    'Merchant': merchants,
    'Category': categories,
    'Account': accounts,
    'Original Statement': original_statements,
    'Notes': notes,
    'Amount': amounts
}

df = pd.DataFrame(data)
df


## Statement Dataframe Clean Up


In [ ]:
#Drop any row with placeholder value "1234" or "" in date column (those rows are not transactions)
df = df.loc[(df["Date"] != "1234") & (df["Date"] != "")]

#Set date as index because number index is out of order when rows were dropped
df = df.set_index('Date')
df.tail(20)

In [ ]:
#Add year 2024 to the date column
df['Date'] = df['Date'] + '/2024'

In [ ]:
#fill in Account column with name of bank
df["Account"] = "TD Bank"
df.head()

In [ ]:
#Function to update the category based on the merchant information. Add more conditions as needed.
#This is a simple example and may need to be updated based on your data.
def update_category(merchant):
    if "Amazon" in merchant:
        return "Shopping"
    elif "UBER EATS" in merchant:
        return "Take Out"
    elif "Soho House" in merchant:
        return "Restaurant"
    elif "CHATGPT" in merchant:
        return "Business"
    elif "NAIL" in merchant:
        return "Personal Care"
    elif "RTP RCVD" in merchant:
        return "Paycheck (Coaching)"
    elif "WIX" in merchant:
        return "Business"
    elif "Walgreens" in merchant:
        return "Personal Care"
    elif "FEE" in merchant:
        return "Fees"
    elif "CLASSPASS" in merchant:
        return "Exercise"
    elif "TRADER" in merchant:
        return "Groceries"   
    elif "grocery" in merchant:
        return "Groceries"
    elif "parking" in merchant:
        return "Car Expense"
    elif "UBER" in merchant:
        return "Transportation"
    elif "LYFT" in merchant:
        return "Transportation"
    elif "Zoom" in merchant:
        return "Business"
    elif "ZOOMVIDEOCO" in merchant:
        return "Business"
    elif "BILT" in merchant:
        return "Rent"
    elif "SEPHORA" in merchant:
        return "Shopping"
    elif "GYM" in merchant:
        return "Exercise"
    elif "YOGA" in merchant:
        return "Exercise"
    elif "ATT PAYMENT" in merchant:
        return "Utilities"
    elif "AMEX" in merchant:
        return "CC Payment"
    elif "CAPITAL ONE" in merchant:
        return "CC Payment"
    elif "AFFIRM" in merchant:
        return "Pay Over Time Repayment" 
    elif "FPL" in merchant:
        return "Utilities"
    elif "ERENTERPLAN" in merchant:
        return "Utilities"
    elif "SLACK" in merchant:
        return "Business"
    elif "SPOTIFYUSAI" in merchant:
        return "Subscription"
    elif "ELLEVEST" in merchant:
        return "Subscription"
    elif "AUDIBLE" in merchant:
        return "Subscription"
    else:
        return "Other"

# Apply the function to update the 'Category' column
df['Category'] = df['Merchant'].apply(update_category)

# Display the updated DataFrame
df.head()

In [ ]:
# Visually see the distribution of categories
df["Category"].value_counts()

In [ ]:
#See the unique values in the 'Category' column to determine if any further category consolidation is needed above
# Filter the DataFrame to get rows where the 'Category' is 'other'
filtered_df = df[df['Category'] == 'Other']

# Get the unique values in the 'Merchant' column for these rows
unique_merchants = filtered_df['Merchant'].unique()

# Display the unique merchants
print("Unique merchants in 'other' category:", unique_merchants)

In [ ]:
df.reset_index(inplace=True)
# Show DF first 20 rows to determine where deposit ends and expenses start
# In next step expenses will be given negative value. 
df.head(20)

In [ ]:
# on Monarch Money where these data will be upload, the amount column should be negative for expenses. However, expenses with 4 digits that had a comma value needed further processing to remove special character and then add the negative sign. 
# Clean the 'Amount' column to remove any special characters and whitespace
df['Amount'] = df['Amount'].str.replace(',', '').str.replace('$', '').str.strip()

# Convert the 'Amount' column back to numeric values
df['Amount'] = pd.to_numeric(df['Amount'], errors='coerce')

# Add a negative sign to the 'Amount' column for rows 14 to end
df.loc[11:, 'Amount'] = -df.loc[11:, 'Amount'].abs()

# Display the modified DataFrame for verification
df.head(20)

In [ ]:
#Check the total add the end of the csv matches the total in the PDF statement

total = df["Amount"].sum()
previous_balance = ** # Enter the previous balance in your accoun
final_balance = total + previous_balance
final_balance

In [ ]:
#Lastly im cleaning up the merchant column to remove Date and other numbers that are in the original statement.

# Function to remove numbers and special characters from a string using string methods
def remove_numbers(text):
    # Create a list of characters to remove
    remove_chars = set('0123456789,.*/')
    # Create a new string excluding the characters in remove_chars
    return ''.join(char for char in text if char not in remove_chars)

# Apply the function to the 'Merchant' column
df['Merchant'] = df['Merchant'].apply(remove_numbers)


df.head()

In [ ]:
# Save to CSV
csv_path = '/Users/priscillamorales/Desktop/CodeSpace/Finances/transaction_april.csv'
df.to_csv(csv_path, index=False)

csv_path

## Visualize Category Data
 

In [ ]:
#Function to summarize by category including income and expenses
def summarize_spending(df):
    summary = df.groupby('Category')['Amount'].sum().reset_index()
    return summary

#Apply the function to get amount spent on each category
category_summary = summarize_spending(df)

#View the category summary
category_summary

In [ ]:
#Create a new DataFrame for expenses only
expenses_df = df[df['Amount'] < 0]

# Apply the summarize_spending function to expenses only 
expenses_df = summarize_spending(expenses_df)
expenses_df

In [ ]:
#Create pie chart for visual representation of spending by category
#Expenses are noted in df as negative values, thus the absolute value is taken to represent the pie chart
plt.figure(figsize=(8, 8))
plt.pie(expenses_df['Amount'].abs(), labels=expenses_df['Category'], autopct='%1.1f%%', startangle=200)
plt.title('Spending by Category')
plt.axis('equal')  # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()